# 휴먼지능정보공학과 201810808 정민지

## 연습문제

주의:

 - 다른 과제보다 많이 약 10% 배점합니다

 - 다른 학생과 협력하지 마세요.

 - 제한시간이 다른 과제에 비해 짧아요 (3.5일)



A, B 2인의 가위바위보 후 내기금액을 이긴 사람에게 지급하는 게임을 블록체인에 개발하세요.

게임 컨트랙은 Rsp, 구현할 함수는 다음과 같다. 그 외 필요로 하는 함수는 추가할 수 있다.



함수 | 설명
-----|-----
setA | 무작위로 가위, 바위, 보를 생성하고, 설정한다. ```1000 wei```씩 금액을 건다.
setB | 외부에서 직접 입력하여 설정한다. ```1000 wei```씩 금액을 건다.
play | 가위, 바위, 보의 승패를 정한다.
distributeBetAmount | 이긴 측에게 내기 금액을 분배한다. A가 이기면 B의 내기 금액 ```1000 wei```를 가지게 된다.
getMatchResult() | "A wins", "B wins", "tie" 결과를 출력



노드에서 게임을 다음과 같이 진행하세요.

- 게임 전의 A잔고, B잔고, 컨트랙 잔고 출력

- A가 내기금액 걸고, 컴퓨터에서 무작위로 선택한 가위바위보

- B가 내기금액 걸고, 사용자가 입력한 가위바위보

- 승부내기

- 누가 이겼는지 출력

- 게임 끝나고 A잔고, B잔고, 컨트랙 잔고 출력

In [2]:
%%writefile src/Rsp.sol
pragma solidity 0.5.1;
contract Rsp{
    struct Player {
        string name;
        uint batting;
        uint setRsp;
    }
    int256 result=-1;
    address payable owner;
    address addA;
    address addB;
    uint balance;
    constructor() public {
        owner = msg.sender;
    }
    
    mapping(address=>Player) playerMap;
    mapping(string=>address) addressByName;
    mapping (address => uint) balanceOf;
    
    function setAddressAB(address receiver)public{
        addA=address(this);
        addB=receiver;
    }

    function deposit(uint amount) payable public onlyOwner { 
        require(msg.value == amount);
        balanceOf[addA]+=amount;
        balanceOf[addB]+=amount;
    }
    
    function setA() public{
        uint _rsp= uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
        Player memory m=Player("Player_A",1000,_rsp);
        playerMap[addA]=m;
        addressByName["Player_A"]=addA;
    }
    
    function setB(uint _batting, uint _rsp) public{
        Player memory m2=Player("Player_B",_batting,_rsp);
        playerMap[addB]=m2;
        addressByName["Player_B"]=addB;
    }
    
    function play() public{
        //0:주먹 1:가위: 2:보
        string memory name1="Player_A";
        string memory name2="Player_B";
        Player memory u1=playerMap[addressByName[name1]];
        Player memory u2=playerMap[addressByName[name2]];
        uint a=u1.setRsp;
        uint b=u2.setRsp;
        if(a==b){
            result=0;//tie
        }
        else if((a==0&&b==1)||(a==1&&b==2)||(a==2&&b==0)){
            result=1;//a wins
        }
        else{
            result=2;//b wins
        }
    }
    
    function getMatchResult() public view returns (string memory){
        if(result==1){
            return("A wins");
        }
        else if(result==2){
            return("B wins");
        }
        else{
            return("tie");
        }
    }
    function distributeBetAmount() public payable{
        string memory name1="Player_A";
        string memory name2="Player_B";
        Player memory u1=playerMap[addressByName[name1]];
        Player memory u2=playerMap[addressByName[name2]];
        uint a_batting=u1.batting;
        uint b_batting=u2.batting;
        if(result==1){
            balanceOf[addressByName[name2]]-=b_batting;
            balanceOf[addressByName[name1]]+=b_batting;
        }
        else if(result==2){
            balanceOf[addressByName[name2]]+=a_batting;
            balanceOf[addressByName[name1]]-=a_batting;
        }
    }
    
    function getBalance() public view returns(uint, uint, uint) {
        return ( balanceOf[addA], balanceOf[addB],balanceOf[address(this)]);
    }
    
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
}

Overwriting src/Rsp.sol


In [3]:
!solc --gas --abi --bin src/Rsp.sol


======= src/Rsp.sol:Rsp =======
Gas estimation:
construction:
   41432 + 1115000 = 1156432
external:
   deposit(uint256):	41741
   distributeBetAmount():	infinite
   getBalance():	1605
   getMatchResult():	infinite
   play():	infinite
   setA():	infinite
   setAddressAB(address):	40816
   setB(uint256,uint256):	infinite
Binary: 
60806040527fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff60005534801561003457600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506115c7806100856000396000f3fe608060405260043610610088576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe01461008d5780631fc376f7146100c65780634a686381146100dd578063504e31f21461016d57806393e84cd9146101be578063a0be28f5146101d5578063b6b55f25146101df578063f31805461461020d575b600080fd5b34801561009957600080fd5b506100a2610252565b60405180848152602001838152602001828152602001935050505060405180910390f35b3480

In [1]:
%%writefile src/RspDeploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray =[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"receiver","type":"address"}],"name":"setAddressAB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_batting","type":"uint256"},{"name":"_rsp","type":"uint256"}],"name":"setB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="0x" + "60806040527fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff60005534801561003457600080fd5b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506115c7806100856000396000f3fe608060405260043610610088576000357c01000000000000000000000000000000000000000000000000000000009004806312065fe01461008d5780631fc376f7146100c65780634a686381146100dd578063504e31f21461016d57806393e84cd9146101be578063a0be28f5146101d5578063b6b55f25146101df578063f31805461461020d575b600080fd5b34801561009957600080fd5b506100a2610252565b60405180848152602001838152602001828152602001935050505060405180910390f35b3480156100d257600080fd5b506100db610366565b005b3480156100e957600080fd5b506100f2610542565b6040518080602001828103825283818151815260200191508051906020019080838360005b83811015610132578082015181840152602081019050610117565b50505050905090810190601f16801561015f5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b34801561017957600080fd5b506101bc6004803603602081101561019057600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610610565b005b3480156101ca57600080fd5b506101d3610695565b005b6101dd610abd565b005b61020b600480360360208110156101f557600080fd5b81019080803590602001909291905050506111f4565b005b34801561021957600080fd5b506102506004803603604081101561023057600080fd5b81019080803590602001909291908035906020019092919050505061133f565b005b600080600060076000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205460076000600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054600760003073ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054925092509250909192565b600060034244604051602001808381526020018281526020019250505060405160208183030381529060405280519060200120600190048115156103a657fe5b0660ff1690506103b46114d4565b6060604051908101604052806040805190810160405280600881526020017f506c617965725f4100000000000000000000000000000000000000000000000081525081526020016103e881526020018381525090508060056000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008201518160000190805190602001906104879291906114f6565b506020820151816001015560408201518160020155905050600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16600660405180807f506c617965725f410000000000000000000000000000000000000000000000008152506008019050908152602001604051809103902060006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055505050565b60606001600054141561058c576040805190810160405280600681526020017f412077696e730000000000000000000000000000000000000000000000000000815250905061060d565b600260005414156105d4576040805190810160405280600681526020017f422077696e730000000000000000000000000000000000000000000000000000815250905061060d565b6040805190810160405280600381526020017f746965000000000000000000000000000000000000000000000000000000000081525090505b90565b30600260006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555080600360006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555050565b60606040805190810160405280600881526020017f506c617965725f41000000000000000000000000000000000000000000000000815250905060606040805190810160405280600881526020017f506c617965725f4200000000000000000000000000000000000000000000000081525090506107116114d4565b600560006006856040518082805190602001908083835b60208310151561074d5780518252602082019150602081019050602083039250610728565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060606040519081016040529081600082018054600181600116156101000203166002900480601f0160208091040260200160405190810160405280929190818152602001828054600181600116156101000203166002900480156108805780601f1061085557610100808354040283529160200191610880565b820191906000526020600020905b81548152906001019060200180831161086357829003601f168201915b505050505081526020016001820154815260200160028201548152505090506108a76114d4565b600560006006856040518082805190602001908083835b6020831015156108e357805182526020820191506020810190506020830392506108be565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060606040519081016040529081600082018054600181600116156101000203166002900480601f016020809104026020016040519081016040528092919081815260200182805460018160011615610100020316600290048015610a165780601f106109eb57610100808354040283529160200191610a16565b820191906000526020600020905b8154815290600101906020018083116109f957829003601f168201915b5050505050815260200160018201548152602001600282015481525050905060008260400151905060008260400151905080821415610a5b5760008081905550610ab5565b600082148015610a6b5750600181145b80610a825750600182148015610a815750600281145b5b80610a995750600282148015610a985750600081145b5b15610aab576001600081905550610ab4565b60026000819055505b5b505050505050565b60606040805190810160405280600881526020017f506c617965725f41000000000000000000000000000000000000000000000000815250905060606040805190810160405280600881526020017f506c617965725f420000000000000000000000000000000000000000000000008152509050610b396114d4565b600560006006856040518082805190602001908083835b602083101515610b755780518252602082019150602081019050602083039250610b50565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060606040519081016040529081600082018054600181600116156101000203166002900480601f016020809104026020016040519081016040528092919081815260200182805460018160011615610100020316600290048015610ca85780601f10610c7d57610100808354040283529160200191610ca8565b820191906000526020600020905b815481529060010190602001808311610c8b57829003601f168201915b50505050508152602001600182015481526020016002820154815250509050610ccf6114d4565b600560006006856040518082805190602001908083835b602083101515610d0b5780518252602082019150602081019050602083039250610ce6565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060606040519081016040529081600082018054600181600116156101000203166002900480601f016020809104026020016040519081016040528092919081815260200182805460018160011615610100020316600290048015610e3e5780601f10610e1357610100808354040283529160200191610e3e565b820191906000526020600020905b815481529060010190602001808311610e2157829003601f168201915b505050505081526020016001820154815260200160028201548152505090506000826020015190506000826020015190506001600054141561102f5780600760006006886040518082805190602001908083835b602083101515610eb75780518252602082019150602081019050602083039250610e92565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254039250508190555080600760006006896040518082805190602001908083835b602083101515610f8f5780518252602082019150602081019050602083039250610f6a565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055506111ec565b600260005414156111eb5781600760006006886040518082805190602001908083835b6020831015156110775780518252602082019150602081019050602083039250611052565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555081600760006006896040518082805190602001908083835b60208310151561114f578051825260208201915060208101905060208303925061112a565b6001836020036101000a038019825116818451168082178552505050505050905001915050908152602001604051809103902060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825403925050819055505b5b505050505050565b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561125057600080fd5b803414151561125e57600080fd5b8060076000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055508060076000600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555050565b6113476114d4565b6060604051908101604052806040805190810160405280600881526020017f506c617965725f4200000000000000000000000000000000000000000000000081525081526020018481526020018381525090508060056000600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008201518160000190805190602001906114189291906114f6565b506020820151816001015560408201518160020155905050600360009054906101000a900473ffffffffffffffffffffffffffffffffffffffff16600660405180807f506c617965725f420000000000000000000000000000000000000000000000008152506008019050908152602001604051809103902060006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550505050565b6060604051908101604052806060815260200160008152602001600081525090565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061153757805160ff1916838001178555611565565b82800160010185558215611565579182015b82811115611564578251825591602001919060010190611549565b5b5090506115729190611576565b5090565b61159891905b8082111561159457600081600090555060010161157c565b5090565b9056fea165627a7a7230582005d11c36834d1c704622cb43785b9d8c7a0ac7c6037d9455fbe8d9dab152855e0029";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 2000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })

    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/RspDeploy.js


In [119]:
!node src/RspDeploy.js

Deploying the contract from 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
hash: 0x2fddf4d48228b5fb7ec24604358a92a2df83b6a5f76e67b273085a4661562d60
---> The contract deployed to: 0xf9EEf55036206a18863527901332d00cb6025b38


In [121]:
!geth --exec "eth.getTransactionReceipt('0x2fddf4d48228b5fb7ec24604358a92a2df83b6a5f76e67b273085a4661562d60')" attach http://localhost:8445

{
  blockHash: "0xcba9613f172b6fa9ecb6246440d1129c3ed0e14b4302a86c2da28bd0c333cf4e",
  blockNumber: 565,
  contractAddress: "0xf9eef55036206a18863527901332d00cb6025b38",
  cumulativeGasUsed: 1566984,
  from: "0x8e71085ffefccb88a4b54d4c93a866db7928a486",
  gasUsed: 1566984,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  root: "0x232c74a21248c7a4543667030c19243752dba7b339d409986be9f41b77754592",
  to: null,
  transactionHash: "0x2fddf4d48228b5fb7ec24604358a92a2df83b6a5f76e67b273085a4661562d60",
  tr

### 매번 호출때마다 시작시 10000씩 충전하고 시작한다.

In [122]:
%%writefile src/RspUse.js
var Web3=require('web3');
var fs=require('fs');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
}
var _abiArray =[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"receiver","type":"address"}],"name":"setAddressAB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_batting","type":"uint256"},{"name":"_rsp","type":"uint256"}],"name":"setB","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];

async function doIt() {
    var rsp= new web3.eth.Contract(_abiArray, '0xf9EEf55036206a18863527901332d00cb6025b38');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    
    //계정 A,B 설정
    await rsp.methods.setAddressAB(accounts[1]).send({from: accounts[0], gas: 122209});
    
    //A,B에 10000wei 충전
    await rsp.methods.deposit(10000).send({from: accounts[0], gas: 122209,value:10000});
    
    //초기 금액 출력
    rsp.methods.getBalance().call().then(function(value) {console.log("<Init balance>\nA:", value[0], "  B:", value[1], "  Contract:",value[2],"\n")});
    
    //setA
    await rsp.methods.setA().send({from: accounts[0], gas: 122209});
    
    //setB, 내기 금액과 가위바위보중 선택(0: 주먹, 1:가위, 2:보자기)
    await rsp.methods.setB(1000,2).send({from: accounts[0], gas: 122209});
    
    //비교
    await rsp.methods.play().send({from: accounts[0], gas: 122209});
    
    //결과 출력
    rsp.methods.getMatchResult().call().then(console.log);
    
    //결과에 따른 금액 배정
    await rsp.methods.distributeBetAmount().send({from: accounts[0], gas: 122209});
    
    //금액까지 옮겨진 결과
    rsp.methods.getBalance().call().then(function(value) {console.log("\n<game After>\nA:", value[0], "  B:", value[1], "  Contract:",value[2])});
    
}

doIt();


Overwriting src/RspUse.js


### A가 이겼을 때

In [123]:
!node src/RspUse.js

Account: 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
<Init balance>
A: 10000   B: 10000   Contract: 10000 

A wins

<game After>
A: 11000   B: 9000   Contract: 11000


In [124]:
!node src/RspUse.js

Account: 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
<Init balance>
A: 21000   B: 19000   Contract: 21000 

A wins

<game After>
A: 22000   B: 18000   Contract: 22000


### 비겼을 때

In [125]:
!node src/RspUse.js

Account: 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
<Init balance>
A: 32000   B: 28000   Contract: 32000 

tie

<game After>
A: 32000   B: 28000   Contract: 32000


In [126]:
!node src/RspUse.js

Account: 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
<Init balance>
A: 42000   B: 38000   Contract: 42000 

tie

<game After>
A: 42000   B: 38000   Contract: 42000


In [127]:
!node src/RspUse.js

Account: 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
<Init balance>
A: 52000   B: 48000   Contract: 52000 

tie

<game After>
A: 52000   B: 48000   Contract: 52000


In [128]:
!node src/RspUse.js

Account: 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
<Init balance>
A: 62000   B: 58000   Contract: 62000 

tie

<game After>
A: 62000   B: 58000   Contract: 62000


In [129]:
!node src/RspUse.js

Account: 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
<Init balance>
A: 72000   B: 68000   Contract: 72000 

A wins

<game After>
A: 73000   B: 67000   Contract: 73000


### b가 이겼을 때

In [130]:
!node src/RspUse.js

Account: 0x8e71085FfEFcCb88a4B54D4c93A866DB7928A486
<Init balance>
A: 83000   B: 77000   Contract: 83000 

B wins

<game After>
A: 82000   B: 78000   Contract: 82000
